# [廖雪峰教程——调试](https://www.liaoxuefeng.com/wiki/1016959663602400/1017602696742912)


程序能一次写完并正常运行的概率很小，基本不超过1%。总会有各种各样的bug需要修正。有的bug很简单，看看错误信息就知道，有的bug很复杂，我们需要知道出错时，哪些变量的值是正确的，哪些变量的值是错误的，因此，需要一整套调试程序的手段来修复bug。

### print
第一种方法简单直接粗暴有效，就是用print()把可能有问题的变量打印出来看看：

In [1]:
def foo(s):
    n = int(s)
    print('>>> n = %d' % n)
    return 10 / n

def main():
    foo('0')

main()

>>> n = 0


ZeroDivisionError: division by zero

执行后在输出中查找打印的变量值，用print()最大的坏处是将来还得删掉它，想想程序里到处都是print()，运行结果也会包含很多垃圾信息。所以，我们又有第二种方法。

### 断言
凡是用print()来辅助查看的地方，都可以用**断言（assert）**来替代：

In [3]:
def foo(s):
    n = int(s)
    assert n != 0, 'n is zero!'
    return 10 / n

def main():
    foo('0')
    
main()

AssertionError: n is zero!

assert的意思是，表达式n != 0应该是True，否则，根据程序运行的逻辑，后面的代码肯定会出错。

如果断言失败，assert语句本身就会抛出AssertionError。

程序中如果到处充斥着assert，和print()相比也好不到哪去。不过，启动Python解释器时可以用-O参数来关闭assert：

```
$ python -O err.py
Traceback (most recent call last):
  ...
ZeroDivisionError: division by zero
```

**注意：断言的开关“-O”是英文大写字母O，不是数字0。** 关闭后，你可以把所有的assert语句当成pass来看。

### logging

把print()替换为logging是第3种方式，和assert比，logging不会抛出错误，而且可以输出到文件：

In [4]:
import logging

s = '0'
n = int(s)
logging.info('n = %d' % n)
print(10 / n)

ZeroDivisionError: division by zero

logging.info()就可以输出一段文本。运行，发现除了ZeroDivisionError，没有任何信息。怎么回事？

别急，在import logging之后添加一行配置再试试：

```
import logging
logging.basicConfig(level=logging.INFO)

s = '0'
n = int(s)
logging.info('n = %d' % n)
print(10 / n)
```

看到输出了：

```
$ python err.py
INFO:root:n = 0
Traceback (most recent call last):
  File "err.py", line 8, in <module>
    print(10 / n)
ZeroDivisionError: division by zero
```

这就是logging的好处，它允许你指定记录信息的级别，有debug，info，warning，error等几个级别，当我们指定level=INFO时，logging.debug就不起作用了。同理，指定level=WARNING后，debug和info就不起作用了。这样一来，你可以放心地输出不同级别的信息，也不用删除，最后统一控制输出哪个级别的信息。

logging的另一个好处是通过简单的配置，一条语句可以同时输出到不同的地方，比如console和文件。

### pdb
 
第4种方式是启动Python的调试器pdb(python debug)，让程序以单步方式运行，可以随时查看运行状态。我们先准备好程序：

```
# err.py

s = '0'
n = int(s)
print(10 / n)
```

以参数-m pdb启动后，**pdb定位到下一步要执行的代码**-> s = '0'。

- 输入命令`l`来查看代码
- 输入命令`n`可以单步执行代码
- 输入命令`p 变量名`来查看变量
- 输入命令`q`结束调试，退出程序

In [2]:
%run -m pdb err.py

> /Users/xusanshan/learngit/Topics/err.py(6)<module>()
-> s = '0'
(Pdb) l
  1  	#import logging
  2  	#logging.basicConfig(level=logging.INFO)
  3  	#logging.basicConfig(level=logging.WARNING)
  4  	#import pdb
  5  	
  6  ->	s = '0'
  7  	n = int(s)
  8  	#logging.info('n = %d' % n)
  9  	#pdb.set_trace()
 10  	print(10 / n)
[EOF]
(Pdb) n
> /Users/xusanshan/learngit/Topics/err.py(7)<module>()
-> n = int(s)
(Pdb) n
> /Users/xusanshan/learngit/Topics/err.py(10)<module>()
-> print(10 / n)
(Pdb) p n
0
(Pdb) q


这种通过pdb在命令行调试的方法理论上是万能的，但实在是太麻烦了，如果有一千行代码，要运行到第999行得敲多少命令啊。还好，我们还有另一种调试方法。

### pdb.set_trace()
这个方法也是用pdb，但是不需要单步执行，我们只需要import pdb，然后，在可能出错的地方放一个pdb.set_trace()，就可以设置一个断点：

```
# err.py
import pdb

s = '0'
n = int(s)
pdb.set_trace() # 运行到这里会自动暂停
print(10 / n)
```

不需要参数运行代码，程序会自动在pdb.set_trace()暂停并进入pdb调试环境，可以用命令p查看变量，或者用命令c继续运行：

In [4]:
%run err.py

> /Users/xusanshan/learngit/Topics/err.py(10)<module>()
-> print(10 / n)
(Pdb) p n
0
(Pdb) c


ZeroDivisionError: division by zero

这个方式比直接启动pdb单步调试效率要高很多，但也高不到哪去。

### IDE
如果要比较爽地设置断点、单步执行，就需要一个支持调试功能的IDE。目前比较好的Python IDE有：

- Visual Studio Code：https://code.visualstudio.com/，需要安装Python插件。
- PyCharm：http://www.jetbrains.com/pycharm/
- 另外，Eclipse加上pydev插件也可以调试Python程序。

### 小结
写程序最痛苦的事情莫过于调试，程序往往会以你意想不到的流程来运行，你期待执行的语句其实根本没有执行，这时候，就需要调试了。虽然用IDE调试起来比较方便，但是最后你会发现，logging才是终极武器。

# [Python中logging模块的基本用法](https://cuiqingcai.com/6080.html)

## 日志记录的重要性

在开发过程中，如果程序运行出现了问题，我们是可以使用我们自己的 Debug 工具来检测到到底是哪一步出现了问题，如果出现了问题的话，是很容易排查的。但程序开发完成之后，我们会将它部署到生产环境中去，这时候代码相当于是在一个黑盒环境下运行的，我们只能看到其运行的效果，是不能直接看到代码运行过程中每一步的状态的。在这个环境下，运行过程中难免会在某个地方出现问题，甚至这个问题可能是我们开发过程中未曾遇到的问题，碰到这种情况应该怎么办？

如果我们现在只能得知当前问题的现象，而没有其他任何信息的话，如果我们想要解决掉这个问题的话，那么只能根据问题的现象来试图复现一下，然后再一步步去调试，这恐怕是很难的，很大的概率上我们是无法精准地复现这个问题的，而且 Debug 的过程也会耗费巨多的时间，这样一旦生产环境上出现了问题，修复就会变得非常棘手。但这如果我们当时有做日志记录的话，不论是正常运行还是出现报错，都有相关的时间记录，状态记录，错误记录等，那么这样我们就可以方便地追踪到在当时的运行过程中出现了怎样的状况，从而可以快速排查问题。

因此，日志记录是非常有必要的，任何一款软件如果没有标准的日志记录，都不能算作一个合格的软件。作为开发者，我们需要重视并做好日志记录过程。

## 日志记录的流程框架

那么在 Python 中，怎样才能算作一个比较标准的日志记录过程呢？或许很多人会使用 print 语句输出一些运行信息，然后再在控制台观察，运行的时候再将输出重定向到文件输出流保存到文件中，这样其实是非常不规范的，在 Python 中有一个标准的 logging 模块，我们可以使用它来进行标注的日志记录，利用它我们可以更方便地进行日志记录，同时还可以做更方便的级别区分以及一些额外日志信息的记录，如时间、运行模块信息等。

接下来我们先了解一下日志记录流程的整体框架。
